# Problem
How much does Wikipedia editors stray from their domain/topic expertise?
Let each articles be represented with an ordered list of embeddings $$(a_1, a_2,..., a_m)$$
and two articles' semantic difference measured by their L2 norm $$dist(x,y) = \sqrt{\sum_{i=1}^m (x_i - y_i)^2}$$

Let each editor be represented with a list of articles that they edit and how many times they edit them $$((a_1, c_1), (a_2, c_2),..., (a_n, c_n))$$
Then, each editor's variance is $$var(x) = \sqrt{\frac{\sum_{i=1}^n dist(a_i, E[x]) * c_i}{\sum_{i=1}^n c_i}}$$

# Hypothesis
Since each editor edits a limited amount of articles, most editors will have variance close to zero. However, editors who edit more articles will likely to have higher variance on their articles.

# Dataset
We use 3 datasets: [Wikipedia XML dump](https://dumps.wikimedia.org/), [Wikipedia HuggingFace dataset](https://huggingface.co/datasets/wikipedia), and [Cohere's Wikipedia semantic embeddings](https://huggingface.co/datasets/Cohere/wikipedia-22-12). All 3 datasets extracts from the [English Wikipedia](https://en.wikipedia.org) and the [Simple English Wikipedia](https://simple.wikipedia.org). The English Wikipedia consist of ~6m articles and ~100k editors. The Simple English Wikipedia consists of ~400k articles.

## Wikipedia XML dump
The XML dump contains articles (article id, text, and their edit history). We parse each XML files and convert it to csv. 

Each article is uniquely identified by their ID (wikipedia-id). Each editor can be uniquely identified by their username, although some editors decide to stay anonymous, in which case their edit will be recorded with their IP. For the sake of consistency, we remove all IP edits.

## Wikipedia HuggingFace dataset
The Huggingface dataset contains articles and their plaintext. We convert each article into a list of tokens and remove all stopword tokens.

## Cohere's Wikipedia embedding
The Cohere's embedding contains articles and their semantic embeddings. Cohere's dataset is aimed towards LLM task, not NLP - each row represented an embeddding of a paragraph inside a Wikipedia article. We remove all paragraph except the first for each article since the first paragraph is likely to be representative of the article.

# Model

## LDA
We use [Latent Dirichlet Allocation](https://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf) to generates semantic embedding for each article. 

LDA assumes the following process to generate a document:
1. Choose the length of the document $N \sim Poisson(\xi)$. The distribution doesn't have to be Poisson - it can be any probability distribution.
2. Choose the topic mixture $\theta \sim Dir(\alpha)$
3. For each of the N words $w_i$:

    a) Choose a topic $z_i \sim Multinomial(\theta)$. 
    
    b) Choose a word $w_i \sim p(w_i | z_i, \beta)$, where $\beta$ is the topic-word probability matrix where $\beta_{ij} = p(w_j = 1 | z_i = 1)$. 
    
The goal is to maximize the posterior distribution $$\Pr(\theta, z | w, \alpha, \beta) = \frac{\Pr(\theta, z, w | \alpha, \beta)}{\Pr(w | \alpha, \beta)}$$ that is, the probability of the topic mixture and the word mixture given a document.
    
Note that the model assumes that words are i.i.d. random variables - words are generated independently and identically distributed, and there's no ordering to words (effectively treating a document as a multiset of words). Since LDA, there have been advances in NLP such as Transformer model (BERT) that is able to take word's location in the sentence/paragraph into account.

The model's performance is measured by its log likehood of the seen dataset and its perplexity. The log likelihood measures  how likely is it that this corpus is generated given the model. The perplexity, which is inversely correlated with log likelihood, is equivalent to the inverse of the geometric mean of per-word likelihood (the likelihood of each word being generated).

## Other model
We considered unigram and tf-idf model. 

Unigram is a simplified version of LDA where you assume all the words come from the same probability distribution, without considering topics, and this probability distribution is simply the fraction of time the word appears in the corpus. 

TF-IDF calculate the weight of each word in a certain document by how many times it appears in that document, and how rare it appears in the corpus.

Ultimately, these 2 models are too simplified to effectively model semantic similarity. LDA is the best model for topic modelling and calculating semantic similarity without resorting to pre-trained BERT.

## Business application
Our analysis provides insight into the behaviors of Wikipedia editors. The analysis here should also provide insight into the authenticity and trustworthiness of Wikipedia articles since articles are all written by editors.

# Live system
Our analyses might be deployed as a data pipeline that parse XML dumps once in a while and provide live insights to editors' behavior. We can also provide live query on the behavior of any Wikipedia editor.

The end model would be available as a database with an API endpoint, and all data artifacts would be discarded. The end-user would be able to access it through a frontend that interact with the API, or through the API directly.